<a href="https://colab.research.google.com/github/JGrant3101/ME4MachineLearning/blob/main/Coursework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Will start by importing all the libraries which might need to be used.

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn import datasets
from sklearn import model_selection
from sklearn import naive_bayes
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import pandas
import plotly.graph_objects as go
import sys
import tensorflow
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import *

Firstly going to build the neural network for dataset1

In [37]:
# Start by importing the dataset
dataset1 = pandas.read_csv('/content/dataset1.csv')

# Write each column of the file to an array to make it easier to work with
ArmLength1 = np.array(dataset1['Arm length (m)'][:])
BallWeight1 = np.array(dataset1['Ball weight (kg)'][:])
BallRadius1 = np.array(dataset1['Ball radius (mm)'][:])
AirTemp1 = np.array(dataset1['Air temperature (deg C)'][:])
SpringConst1 = np.array(dataset1['Spring constant (N per m)'][:])
DeviceWeight1 = np.array(dataset1['Device weight (kg)'][:])
TargetHit1 = np.array(dataset1['Target hit'][:])
# Converting our currently 1D y values to 2D to match our model
TargetHit1_binary = to_categorical(TargetHit1)

# Going to scale each all of the data so want to find the mean and standard deviation
ArmLength1Mean = np.mean(ArmLength1)
ArmLength1std = np.std(ArmLength1)
BallWeight1Mean = np.mean(BallWeight1)
BallWeight1std = np.std(BallWeight1)
BallRadius1Mean = np.mean(BallRadius1)
BallRadius1std = np.std(BallRadius1)
AirTemp1Mean = np.mean(AirTemp1)
AirTemp1std = np.std(AirTemp1)
SpringConst1Mean = np.mean(SpringConst1)
SpringConst1std = np.std(SpringConst1)
DeviceWeight1Mean = np.mean(DeviceWeight1)
DeviceWeight1std = np.std(DeviceWeight1)

# Finally creating the scaled arrays using these mean and std values
ArmLength1Scaled = (ArmLength1 - ArmLength1Mean)/ArmLength1std
BallWeight1Scaled = (BallWeight1 - BallWeight1Mean)/BallWeight1std
BallRadius1Scaled = (BallRadius1 - BallRadius1Mean)/BallRadius1std
AirTemp1Scaled = (AirTemp1 - AirTemp1Mean)/AirTemp1std
SpringConst1Scaled = (SpringConst1 - SpringConst1Mean)/SpringConst1std
DeviceWeight1Scaled = (DeviceWeight1 - DeviceWeight1Mean)/DeviceWeight1std

# Finally forming these into one array to be parsed into the fit function
Inputs = np.zeros((2000, 6))
Inputs[:,0] = ArmLength1Scaled
Inputs[:,1] = BallWeight1Scaled
Inputs[:,2] = BallRadius1Scaled
Inputs[:,3] = AirTemp1Scaled
Inputs[:,4] = SpringConst1Scaled
Inputs[:,5] = DeviceWeight1Scaled

Above we have imported and processed our input data to be used to train the network

Now will do the actual construction of the network

In [56]:
# Initialising our neural network
model1 = Sequential()

# Adding layers, these layers are two layers of 4 nodes using the ReLU function and 1 with 2 nodes using the softmax function
model1.add(Dense(units = 10, activation = 'relu', input_dim = 6))
model1.add(Dense(units = 12, activation = 'relu'))
model1.add(Dense(units = 6, activation = 'relu'))
model1.add(Dense(units = 2, activation = 'softmax'))

# Compiling the model
model1.compile(loss = 'categorical_crossentropy', optimizer = 'sgd')

Above is the construction of the neural network, as we have 2000 points I think we can do a 5 split K fold cross validation and get good results so going to run that below in order to see what results the model gives and be able to work on improving it. This method is important to ensure that minimal overfitting is ocurring.

In [ ]:
# Setting up a k fold cross validation
kf = KFold(n_splits = 5, shuffle = True)

# Defining empty arrays to add our fraction values to so an average can be calculated at the end
testfractions = 0
trainfractions = 0
a = 0;

for train_index, test_index in kf.split(Inputs):
  TrainingInputs = Inputs[train_index]
  TrainingTargetHit = TargetHit1_binary[train_index]
  TestInputs = Inputs[test_index]
  TestTargetHit = TargetHit1_binary[test_index]

  # Use X_train, y_train to train the SVM
  model1.fit(TrainingInputs, TrainingTargetHit, epochs = 10, batch_size = 40)
  # Use svm.predict() to predict the output for the test data set
  testresults = model1.predict(TestInputs)
  # loop through to compare the test data output to what it should be and obtain the fraction of correct classifications
  count = 0
  for i in range(len(TestTargetHit[:,0])):
    if (testresults[i, 0] > testresults[i, 1] and TestTargetHit[i, 0] > TestTargetHit[i, 1]) or (testresults[i, 0] < testresults[i, 1] and TestTargetHit[i, 0] < TestTargetHit[i, 1]):
      count = count+1
  testfractions = testfractions + (count / len(TestTargetHit[:,0]))
  # Do the same prediction and assessment performance with the training data
  count = 0
  trainresults = model1.predict(TrainingInputs)
  for i in range(len(TrainingTargetHit[:,0])):
    if (trainresults[i, 0] > trainresults[i, 1] and TrainingTargetHit[i, 0] > TrainingTargetHit[i, 1]) or (trainresults[i, 0] < trainresults[i, 1] and TrainingTargetHit[i, 0] < TrainingTargetHit[i, 1]):
      count = count+1
  trainfractions = trainfractions + (count / len(TrainingTargetHit[:,0]))

# Dividing the sum of 5 fraction values to get the average
testfraction = testfractions/5
trainfraction = trainfractions/5

# Printing the final accuracy values
print(testfraction)
print(trainfraction)

Now model has been tested and proven out can export and run through the check script provided to check everything works and also get an idea of how accurate it is.

In [64]:
# Exporting the model
model1.save('jg619-1.h5')

# Saving the scaling parameters for temperature and strength
scArray = np.array([[ArmLength1Mean, BallWeight1Mean, BallRadius1Mean, AirTemp1Mean, SpringConst1Mean, DeviceWeight1Mean], [ArmLength1std, BallWeight1std, BallRadius1std, AirTemp1std, SpringConst1std, DeviceWeight1std]])
np.savetxt('jg619-1.txt', scArray)

# Downloading the files
from google.colab import files
files.download('jg619-1.h5')
files.download('jg619-1.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>